In [1]:
import numpy as np

# --- оптимизаторы / планировщики / модель ----------------------------------
from core.optimizers import Adam, Nesterov
from core.schedulers import PolynomialDecay
from core.sgd import SGDRegressor

# ---- 1. генерируем/загружаем данные ---------------------------------------
rng = np.random.default_rng(0)
X = rng.standard_normal((1000, 5))
w = rng.standard_normal(5)
y = X @ w + rng.normal(scale=0.1, size=1000)

X_train, X_test = X[:800], X[800:]
y_train, y_test = y[:800], y[800:]

# ---- 2. модель с Adam ------------------------------------------------------
adam_opt = Adam(lr=1e-3, weight_decay=1e-4)

model_adam = SGDRegressor(
    optimizer=adam_opt,
    batch_size=32,
    max_iter=2000,
    init="he",
    clip_norm=5.0,
)
model_adam.fit(X_train, y_train)
print("Adam  R² =", model_adam.score if hasattr(model_adam, 'score') else
      np.corrcoef(model_adam.predict(X_test), y_test)[0, 1]**2)

# ---- 3. модель с Nesterov + PolynomialDecay --------------------------------
nesterov_opt = Nesterov(lr=1e-2, beta=.9)

poly_sched = PolynomialDecay(lr0=1e-2, alpha=.5, beta=1)

model_nest = SGDRegressor(
    optimizer=nesterov_opt,          # если хотим оптимизатор
    lr_schedule=None,                # тогда планировщик не нужен
    batch_size=64,
    max_iter=3000,
)
model_nest.fit(X_train, y_train)

print("Nesterov R² =",
      np.corrcoef(model_nest.predict(X_test), y_test)[0, 1]**2)

# ---- 4. чистый SGD + polynomial scheduler (без внешнего оптимизатора) ------
model_poly = SGDRegressor(
    lr_schedule=poly_sched,
    batch_size=64,
    max_iter=3000,
)
model_poly.fit(X_train, y_train)
print("Poly-Sched R² =",
      np.corrcoef(model_poly.predict(X_test), y_test)[0, 1]**2)


Adam  R² = 0.9936056458803151
Nesterov R² = 0.997627600619082
Poly-Sched R² = 0.9968810997380787


In [2]:
from core.schedulers import StepDecayScheduler
from core.sgd import SGDRegressor
sched = StepDecayScheduler(lr0=1e-2, step=500, gamma=0.5)

model = SGDRegressor(
    lr_schedule=sched,
    penalty="l2",
    alpha=1e-2,
    batch_size=64,
    max_iter=2000,
)
model.fit(X_train, y_train)
print("R² =", model(X_test).var())   # пример вызова


R² = 4.108085927200493
